In [1]:
import pandas as pd
from lib.mutation_util import get_tx_datetime
%load_ext autoreload
%autoreload

In [2]:
df_ori = pd.read_csv('../adyen-dataset.csv')
df_ori["tx_datetime"] = df_ori.apply(get_tx_datetime, axis=1)
df_ori["tx_datetime"] = pd.to_datetime(df_ori["tx_datetime"])
df_init = df_ori.loc(axis=1)[["psp_reference","tx_datetime","has_fraudulent_dispute","is_refused_by_adyen"]]
df_categorical_dummy_features = pd.read_csv(f"./categorical_dummies_features.csv")
df_baseline_features = pd.read_csv(f"./baseline_features.csv")
df_baseline_features = pd.concat([df_baseline_features.loc(axis=1)["is_night":"ip_address_risk_30day_window"],df_baseline_features.loc(axis=1)[["psp_reference","eur_amount"]]], axis=1)
# df_outlier_features = pd.read_csv(f"./outlier_score_July_December.csv")
# df_outlier_features["is_outlier"] = df_outlier_features["outlier_score"]> 0.05
df_graph_features = pd.read_csv("./graph_features_july_december.csv")

In [3]:
df_features = df_init.merge(df_categorical_dummy_features,on="psp_reference")
df_features = df_features.merge(df_baseline_features,on="psp_reference")
# df_features = df_features.merge(df_outlier_features,on="psp_reference", how="left")
df_features = df_features.merge(df_graph_features, on="psp_reference")

In [4]:
df_features.columns

Index(['psp_reference', 'tx_datetime', 'has_fraudulent_dispute',
       'is_refused_by_adyen', 'is_credit', 'no_ip', 'no_email', 'same_country',
       'issuing_country', 'merchant_Merchant B', 'merchant_Merchant C',
       'merchant_Merchant D', 'merchant_Merchant E', 'card_scheme_MasterCard',
       'card_scheme_Other', 'card_scheme_Visa', 'ip_country_GR',
       'ip_country_IT', 'ip_country_NL', 'ip_country_ZW', 'issuing_country_GR',
       'issuing_country_IT', 'issuing_country_NL', 'issuing_country_ZW',
       'device_type_Linux', 'device_type_MacOS', 'device_type_Other',
       'device_type_Windows', 'device_type_iOS', 'shopper_interaction_POS',
       'zip_code_1104', 'zip_code_2039', 'zip_code_3941', 'zip_code_AAD',
       'zip_code_BB', 'zip_code_BZD', 'zip_code_DB', 'zip_code_DFFF',
       'zip_code_EB', 'zip_code_EGHA', 'zip_code_FFR', 'zip_code_FGDD',
       'zip_code_XDED', 'zip_code_XOL', 'zip_code_ZB', 'is_night',
       'is_weekend', 'diff_tx_time_in_hours', 'is_diff_pr

In [5]:
df_features.loc(axis=1)["has_fraudulent_dispute":'same_country'] = df_features.loc(axis=1)["has_fraudulent_dispute":'same_country'].astype(int)
df_features["is_diff_previous_ip_country"] = df_features["is_diff_previous_ip_country"].astype(int)

In [6]:
df_features = df_features.sort_values("tx_datetime")
df_features.to_csv(f"final_features.csv", index=False)

In [7]:
df_features.columns

Index(['psp_reference', 'tx_datetime', 'has_fraudulent_dispute',
       'is_refused_by_adyen', 'is_credit', 'no_ip', 'no_email', 'same_country',
       'issuing_country', 'merchant_Merchant B', 'merchant_Merchant C',
       'merchant_Merchant D', 'merchant_Merchant E', 'card_scheme_MasterCard',
       'card_scheme_Other', 'card_scheme_Visa', 'ip_country_GR',
       'ip_country_IT', 'ip_country_NL', 'ip_country_ZW', 'issuing_country_GR',
       'issuing_country_IT', 'issuing_country_NL', 'issuing_country_ZW',
       'device_type_Linux', 'device_type_MacOS', 'device_type_Other',
       'device_type_Windows', 'device_type_iOS', 'shopper_interaction_POS',
       'zip_code_1104', 'zip_code_2039', 'zip_code_3941', 'zip_code_AAD',
       'zip_code_BB', 'zip_code_BZD', 'zip_code_DB', 'zip_code_DFFF',
       'zip_code_EB', 'zip_code_EGHA', 'zip_code_FFR', 'zip_code_FGDD',
       'zip_code_XDED', 'zip_code_XOL', 'zip_code_ZB', 'is_night',
       'is_weekend', 'diff_tx_time_in_hours', 'is_diff_pr

In [8]:
df_features.head()

,psp_reference,tx_datetime,has_fraudulent_dispute,is_refused_by_adyen,is_credit,no_ip,no_email,same_country,issuing_country,merchant_Merchant B,...,CHScore_MT,CHScore_LT,MerScore,MerScore_ST,MerScore_MT,MerScore_LT,TrxScore,TrxScore_ST,TrxScore_MT,TrxScore_LT
52850,23382223708,2021-07-01 00:05:00,0,1,1,0,0,1,NL,0,...,1.032769e-06,9.551954e-07,0.046098,0.044312,0.046117,0.046098,0.000010,0.000011,0.000010,0.000010
65152,27214733308,2021-07-01 00:25:00,1,0,1,0,0,1,IT,0,...,2.200364e-04,2.091399e-04,0.011669,0.008127,0.011642,0.011668,0.000063,0.000363,0.000065,0.000063
14120,45524281895,2021-07-01 00:35:00,0,0,1,1,0,1,ZW,0,...,7.069965e-07,6.602685e-07,0.046098,0.044312,0.046117,0.046098,0.000008,0.000006,0.000008,0.000008
54618,62692030329,2021-07-01 00:58:00,0,1,1,1,0,1,BR,1,...,2.560043e-04,2.410520e-04,0.023153,0.024819,0.023155,0.023153,0.000068,0.000178,0.000072,0.000068
50265,45361964662,2021-07-01 01:03:00,0,0,0,1,0,1,NL,0,...,2.487407e-06,2.302359e-06,0.022769,0.023104,0.022779,0.022769,0.000009,0.000028,0.000009,0.000009
